In [2]:
import json
import pandas as pd
from elasticsearch import Elasticsearch
import mariadb
from pyhive import hive

# **1-HIVE**

Vamos a sacar todas las victorias de Fernando Alonso, y deberia de devolvernos 32 resultados.

In [3]:
conn = hive.Connection(host="localhost", username="hive", database="f1_hive")
cursor = conn.cursor()

In [4]:
cursor.execute('''
SELECT rs.name, rs.year
FROM hive_results r
JOIN hive_drivers d ON r.driverid = d.driverid
JOIN hive_races rs ON r.raceid = rs.raceid
WHERE d.driverref = 'alonso'
  AND r.position = 1
               ''')

results = cursor.fetchall()
print(results)
print(len(results))

[('"Singapore Grand Prix"', 2008), ('"Japanese Grand Prix"', 2008), ('"Malaysian Grand Prix"', 2007), ('"Monaco Grand Prix"', 2007), ('"European Grand Prix"', 2007), ('"Italian Grand Prix"', 2007), ('"Bahrain Grand Prix"', 2006), ('"Australian Grand Prix"', 2006), ('"Spanish Grand Prix"', 2006), ('"Monaco Grand Prix"', 2006), ('"British Grand Prix"', 2006), ('"Canadian Grand Prix"', 2006), ('"Japanese Grand Prix"', 2006), ('"Malaysian Grand Prix"', 2005), ('"Bahrain Grand Prix"', 2005), ('"San Marino Grand Prix"', 2005), ('"European Grand Prix"', 2005), ('"French Grand Prix"', 2005), ('"German Grand Prix"', 2005), ('"Chinese Grand Prix"', 2005), ('"Hungarian Grand Prix"', 2003), ('"Bahrain Grand Prix"', 2010), ('"German Grand Prix"', 2010), ('"Italian Grand Prix"', 2010), ('"Singapore Grand Prix"', 2010), ('"Korean Grand Prix"', 2010), ('"British Grand Prix"', 2011), ('"Malaysian Grand Prix"', 2012), ('"European Grand Prix"', 2012), ('"German Grand Prix"', 2012), ('"Chinese Grand Prix"

In [5]:
cursor.close()
conn.close()

# **2-ELASTICSEARCH**

In [ ]:
es = Elasticsearch("http://localhost:9200")

res = es.search(index='my_index', body={
     'query': {
         'range': {
             'position': {
                 'gte': 1,
                 'lte': 3
             }
         }
     },
     'size': 1
})

# **3-GRAPHDB**